## Syntax-dependent trace



In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
CACHE_DIR = "/scratch/lucchetti.f/models/"
os.environ['TRANSFORMERS_CACHE'] = CACHE_DIR


import sys
sys.path.append('..')

import gc
import torch
from model_utils import *
import tqdm as notebook_tqdm
from model import ModelLoader

# from model_loader import ModelLoader
# from model_manager import ModelManager
# from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig

#MODEL_NAME = "Salesforce/codegen-16B-mono"
MODEL_NAME = "bigcode/santacoder"

check_devs()

/home/lucchetti.f/.local/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0 / 34079899648 used for device 0, reserved 0


In [3]:
lm = ModelLoader(MODEL_NAME)
lm.model

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


Load time: 515336719859 ns
transformer.wte.weight torch.Size([49280, 2048]) cuda:0
transformer.wpe.weight torch.Size([2048, 2048]) cuda:0
transformer.h.0.ln_1.weight torch.Size([2048]) cuda:0
transformer.h.0.ln_1.bias torch.Size([2048]) cuda:0
transformer.h.0.attn.q_attn.weight torch.Size([2048, 2048]) cuda:0
transformer.h.0.attn.q_attn.bias torch.Size([2048]) cuda:0
transformer.h.0.attn.kv_attn.weight torch.Size([2048, 256]) cuda:0
transformer.h.0.attn.kv_attn.bias torch.Size([256]) cuda:0
transformer.h.0.attn.c_proj.weight torch.Size([2048, 2048]) cuda:0
transformer.h.0.attn.c_proj.bias torch.Size([2048]) cuda:0
transformer.h.0.ln_2.weight torch.Size([2048]) cuda:0
transformer.h.0.ln_2.bias torch.Size([2048]) cuda:0
transformer.h.0.mlp.c_fc.weight torch.Size([2048, 8192]) cuda:0
transformer.h.0.mlp.c_fc.bias torch.Size([8192]) cuda:0
transformer.h.0.mlp.c_proj.weight torch.Size([8192, 2048]) cuda:0
transformer.h.0.mlp.c_proj.bias torch.Size([2048]) cuda:0
transformer.h.1.ln_1.weight 

GPT2LMHeadCustomModel(
  (transformer): GPT2CustomModel(
    (wte): Embedding(49280, 2048)
    (wpe): Embedding(2048, 2048)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2CustomBlock(
        (ln_1): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2MQAttention(
          (q_attn): Conv1D()
          (kv_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): FastGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2CustomBlock(
        (ln_1): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2MQAttention(
          (q_attn): Conv1D()
          (kv_attn): Conv1D()
          (c_proj): C

In [15]:
lm.extract_fields()

In [16]:
check_devs()

2350448640 / 34079899648 used for device 0, reserved 2365587456


## Inference

In [17]:
model = lm.model
tokenizer = lm.tokenizer

prompts = ["""# intialize three integers
           int x = 1
           int y = 2
           int z ="""]

prompts = ["""int x ="""]

# vanilla generate:

tokenized = tokenizer(prompts, return_tensors="pt")
outputs = model.generate(tokenized.input_ids.to(model.device), max_new_tokens=2, top_k = 1)
tokenizer.decode(outputs[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:49152 for open-end generation.


'int x = 0'

## Trace + activation patch

In [60]:
txt, ret_dict = lm.generate(
    prompts,
    argmax_greedy= True,
    debug = True,
    request_activations= [lm.layer_name_format.format(i) for i in range(5,10)]
)
txt

MASK:  tensor([[1, 1, 1]], device='cuda:0')
MASK:  tensor([[1, 1, 1]], device='cuda:0')
INPUT:  tensor([[389, 727, 256]], device='cuda:0')
ret {'input_tokenized': [[('int', 389), (' x', 727), (' =', 256)]]}
context: slice(0, 3, None)
attn: tensor([[1, 1, 1]], device='cuda:0')
in: tensor([[389, 727, 256]], device='cuda:0')
prompt <0> ==> [(' ', 0.6357), (' (', 0.0573), (' x', 0.0188), (' -', 0.0176), (' i', 0.0134)]
context: slice(0, 3, None)
attn: tensor([[1, 1, 1]], device='cuda:0')
in: tensor([[389, 727, 256]], device='cuda:0')


RuntimeError: The size of tensor a (6) must match the size of tensor b (3) at non-singleton dimension 3

In [ ]:
print_formatted_results(prompts, txt, ret_dict)